In [1]:
import sys
sys.path.append('../')

In [2]:
"""
# Description: Converts the California Department of Environmental Quality's 
#              Prescribed Fire Information Reporting System (PFIRS) dataset 
#              into the Task Force standardized schema.  Dataset
#              is enriched with vegetation, ownership, county, WUI, 
#              Task Force Region, and year.               
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
import os
import datetime
import pandas as pd
from scripts._3a_PFIRS import PFIRS
from scripts.utils import init_gdb, check_schema_lock, og_file_input

workspace, scratch_workspace = init_gdb()

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216



In [3]:
year = str(2023)


project_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
data_path = os.path.join(project_path, year)
lookup_table_path = os.path.join(data_path, "PFIRS_2023\pfirs_agency_2022_2023_AT.xlsx")

if year == '2022':
    pfirs_gdb = os.path.join(data_path, "PFIRS_2022\PFIRS_2022_fromCarlITSgdb.gdb")
    input_fc = os.path.join(pfirs_gdb, "PFIRS_2018_2022") #2022
else:
    pfirs_gdb = os.path.join(data_path, "PFIRS_2023\PFIRS2023.gdb")
    input_fc = os.path.join(pfirs_gdb, "PFIRS2023_20240624Pull") #2023

In [4]:
# INPUTS
# NOTE: Update input dataset as necessary
# pfirs = og_file_input(prefix = 'PFIRS_', filetype = 'Point', gdb = os.path.join(workspace, "b_Originals"))
#pfirs = "PFIRS_2018_2022"
#input_fc = os.path.join(workspace, "b_Originals", pfirs)



# OUTPUTS
# timestamped outputs
output_enriched = os.path.join(workspace, "c_Enriched",f'PFIRS_enriched{year}_{date_id}') 

# REFERENCE (must exist already)
# NOTE: Appended polygon dataset must already exist
prefix = f'Treat_n_harvests_polygons{year}'
treat = og_file_input(prefix = prefix, filetype = 'Polygon', gdb = os.path.join(workspace, "d_Appended"))
treat_poly = os.path.join(workspace, "d_Appended", treat)

check_schema_lock(input_fc)
check_schema_lock(output_enriched)

Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\2023\PFIRS_2023\PFIRS2023.gdb\PFIRS2023_20240624Pull
Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb\c_Enriched\PFIRS_enriched2023_20240927


In [5]:
### seems like the instances need to exist in current running kernel, not in py script
pfirs_lookup_df = pd.read_excel(os.path.join(data_path, "PFIRS_2023\pfirs_agency_2022_2023_AT.xlsx"))
pfirs_lookup_dict = pd.Series(pfirs_lookup_df.agency_calc_field_2.values,index=pfirs_lookup_df.Agency).to_dict()
anonymous_org_list = pfirs_lookup_df[pfirs_lookup_df["Org Type"] == "Industrial Timber"].Agency.tolist()

In [6]:
PFIRS(
    input_fc, 
    lookup_table_path,
    output_enriched, 
    treat_poly
    )

Start Time 2024-09-27 22:09:53.925134
Performing Standardization
   step 1/4 remove some agencies
   step 3/4 rename and add fields
      assigning domains
   step 2/4 import attributes
   remove broadcast burn data for all industry groups except FWS Forestry LLC
copy_features_1:  12225
select_feature_1:  11148
select_feature_2:  1027
point_append:  12175
Saving Output Standardized
standardized has 12175 records
          removing unnecessary fields
        field_list before: ['OBJECTID', 'Shape', 'Burn_ID', 'Burn_Date', 'Air_District', 'COUNTY_', 'Air_Basin', 'Acres_Requested', 'Acres_Approved', 'Acres_Burned', 'Latitude', 'Longitude', 'Legal_Location', 'AGENCY_', 'LMA_Unit', 'LMA_Name', 'Landowner', 'SMP', 'Burn_Unit', 'Burn_Type', 'Fuel_Type', 'Cover_Type', 'Unit_Acres', 'Tons_per_Acre', 'Total_Tons', 'Crossroads', 'Slope', 'Aspect', 'Min_Elevation', 'Max_Elevation', 'Unit_Description', 'Burner_Comments', 'Mitigtations_Contingencies', 'Monitored', 'Monitor_Name', 'Completed', 'Burn_

AttributeError: 'NoneType' object has no attribute 'year'

       enrich step 15/16 Calculating Latitude and Longitude
       enrich step 16/16 removing unnecessary fields
          removing unnecessary fields
        field_list before: ['OBJECTID', 'Shape', 'Latitude', 'Longitude', 'PROJECTID_USER', 'AGENCY', 'ORG_ADMIN_p', 'PROJECT_CONTACT', 'PROJECT_EMAIL', 'ADMINISTERING_ORG', 'PROJECT_NAME', 'PROJECT_STATUS', 'PROJECT_START', 'PROJECT_END', 'PRIMARY_FUNDING_SOURCE', 'PRIMARY_FUNDING_ORG', 'IMPLEMENTING_ORG', 'BatchID_p', 'Val_Status_p', 'Val_MSG_p', 'Val_RunDate_p', 'Review_Status_p', 'Review_MSG_p', 'Review_RunDate_p', 'Dataload_Status_p', 'Dataload_Msg_p', 'TRMTID_USER', 'PROJECTID', 'PROJECTNAME_', 'ORG_ADMIN_t', 'PRIMARY_OWNERSHIP_GROUP', 'PRIMARY_OBJECTIVE', 'SECONDARY_OBJECTIVE', 'TERTIARY_OBJECTIVE', 'TREATMENT_STATUS', 'COUNTY', 'IN_WUI', 'REGION', 'TREATMENT_AREA', 'TREATMENT_START', 'TREATMENT_END', 'RETREATMENT_DATE_EST', 'TREATMENT_NAME', 'BatchID', 'Val_Status_t', 'Val_MSG_t', 'Val_RunDate_t', 'Review_Status_t', 'Review_Msg_t

In [8]:
import geopandas as gpd

In [9]:
pfirs_gdf = gpd.read_file(workspace, driver='OpenFileGDB', layer=os.path.basename(output_enriched))

PROJ: proj_create_from_database: Cannot find proj.db


In [15]:
pfirs_gdf[pfirs_gdf['COUNTS_TO_MAS'] == 'YES'][['ADMINISTERING_ORG', 'ACTIVITY_QUANTITY']].groupby(['ADMINISTERING_ORG']).sum()

,ACTIVITY_QUANTITY
ADMINISTERING_ORG,
California State Parks,8016.890
DOD,20768.330
TIMBER,35384.700
US Department of Energy,14967.645
